In [1]:
import isx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import traceback
import os
import pickle
import shutil
import logging



In [2]:
class pipeline_functions:
    def __init__(self, log_file = None):
        '''nothing required'''
        self.log_file = log_file
    
    
    def print_processing(self, obj, step):
        print(f'ID: {obj.sub_ID} efocus: {obj.efocus} is at step: {step}')
                
    def _dff(self, obj, f0type = 'mean', explicit_fname = None, overwrite = True):
        '''run dff'''
        step = 'dff'
        obj.check_step(step)
        file_out = self.get_output_filepath(obj,step) if not explicit_fname else explicit_fname
        
        if os.path.isfile(file_out):
            if overwrite:
                print(f'overwriting previous file at {file_out}')
                os.remove(file_out)
            else:
                print(f'skipping processing and utilizing previous file at {file_out}')
                return file_out

        file_in = obj.previous_step_filepath
        self.print_processing(obj, step)
        isx.dff(file_in, file_out, f0_type=f0type)
        return file_out
    
    def _preprocess(self, obj, temporal_downsample_factor=2,
                        spatial_downsample_factor=1,
                        crop_rect=None,
                        fix_defective_pixels=True,
                        trim_early_frames=True,
                        explicit_fname = None,
                        overwrite = True):
        '''run preprocess'''
        step = 'preprocess'
        obj.check_step(step)
        file_out = self.get_output_filepath(obj,step) if not explicit_fname else explicit_fname

        if os.path.isfile(file_out):
            if overwrite:
                print(f'overwriting previous file at {file_out}')
                os.remove(file_out)
            else:
                print(f'skipping processing and utilizing previous file at {file_out}')
                return file_out

        file_in = obj.previous_step_filepath
        self.print_processing(obj, step)
        isx.preprocess(file_in, file_out,
                       temporal_downsample_factor=temporal_downsample_factor,
                        spatial_downsample_factor=spatial_downsample_factor,
                        crop_rect=crop_rect,
                        fix_defective_pixels=fix_defective_pixels,
                        trim_early_frames=trim_early_frames)
        return file_out
    
    def _spatial_filter(self, obj, low_cutoff=0.005,
                        high_cutoff=0.5,
                        retain_mean=False,
                        subtract_global_minimum=True,
                        explicit_fname = None,
                        overwrite = True):
        
        '''run spatial filter'''
        step = 'spatial_filter'
        obj.check_step(step)
        file_out = self.get_output_filepath(obj,step) if not explicit_fname else explicit_fname

        if os.path.isfile(file_out):
            if overwrite:
                print(f'overwriting previous file at {file_out}')
                os.remove(file_out)
            else:
                print(f'skipping processing and utilizing previous file at {file_out}')
                return file_out

        file_in = obj.previous_step_filepath
            
        self.print_processing(obj, step)
        
        isx.spatial_filter(file_in, file_out,
                                   low_cutoff=low_cutoff,
                                    high_cutoff=high_cutoff,
                                    retain_mean=retain_mean,
                                    subtract_global_minimum=subtract_global_minimum,)
        return file_out
    
    
    def _check_step(self, obj, step):
        if obj.previous_step_id > obj.parent_pipe.processing_order[step]:
            print(f"""just fyi, youre running things out of the recommended order. your last 
                  step was {obj.order_lookup[obj.previous_step_id]}, and inscopix recommends
                  preprocess -> spatial filter -> motion correct -> dff""")
    
    def get_trim_array(self, obj, indices, keep):
        '''doc string here'''
        
        
        if len(keep) == 0:
            keep = [i for i in range(len(indices)+1)]
            print(f'keep modified to {keep}')

        #get the number of frames. i dont know if the API uses pythonic indexing
        #or if it starts at 1. I will assume it starts at 1.
        num_frames = isx.Movie.read(obj.previous_step_filepath).timing.num_samples

        #assume for now we start and index 1 to get the first frame
        start = [1]
        stop = []
        for index in indices:
            stop += [index]
            start += [index+1]

        stop += [num_frames]
        all_trims = []
        fnames = []
        
        
        for k in keep:
            first= start[k]
            last = stop[k]
            
            step = 'separate_movie'
            #get the base filename
            file_out = self.get_output_filepath(obj, step)   
            
            #modify the filename to include the indexes which it includes
            file_out = self.append_name_general(file_out, f'{first}_to_{last}')
            fnames += [file_out]

            trim_array = []
            if first == 1:
                #remove everything after the stop frame
                trim_array += [[last, num_frames]]

            elif last == num_frames:
                #remove everything from the first frame of the movie to the first frame
                #of this 'crop.' PS I dont like inscopix's nomenclature
                trim_array += [[1, first]]

            else:
                #remove before and after this set of indices
                trim_array+=[[1,first]]
                trim_array+=[[last,num_frames]]

            all_trims += [trim_array]

        return all_trims, fnames
    
    def _pca_ica(self, obj,
                num_pcs=120,
                num_ics=150,
                unmix_type='spatial',
                ica_temporal_weight=0.2,
                max_iterations=100,
                convergence_threshold=1e-05,
                block_size=2000, explicit_fname = None,
                overwrite = True):
        
        step = 'pca_ica'
        obj.check_step(step)
        
        file_out = self.get_output_filepath(obj,step) if not explicit_fname else explicit_fname
    
        
        
        if os.path.isfile(file_out):
            if overwrite:
                print(f'overwriting previous file at {file_out}')
                os.remove(file_out)
            else:
                print(f'skipping processing and utilizing previous file at {file_out}')
                return file_out

        file_in = obj.previous_step_filepath
        
        self.print_processing(obj, step)
        
        isx.pca_ica(input_movie_files=file_in,
                    output_cell_set_files=file_out,
                    num_pcs=num_pcs,
                    num_ics=num_ics,
                    unmix_type=unmix_type,
                    ica_temporal_weight=ica_temporal_weight,
                    max_iterations=max_iterations,
                    convergence_threshold=convergence_threshold,
                    block_size=block_size, 
                     )
        return file_out
        
    
    def _de_interleave(self, obj, overwrite = True):
        step = 'de_interleave'
        planes = obj.get_focal_planes()
        
        file_out = self.get_output_filepath(obj,'')

        fnames_out = [self.append_name_general(file_out, f'efocus_{focus}') for focus in planes]
        try:
            fname_check = [os.path.isfile(file_out) for file in fnames_out]
            
            #do any output files already exist?
            if any(fname_check):
                
                #remove files that exist 
                if overwrite:
                    for file_out in fnames_out:
                        if os.path.isfile(file_out):
                            print(f'overwriting previous file at {file_out}')
                            os.remove(file_out)
                
                #if not overwriting AND all files already exist, return as if processing is complete     
                elif all(fname_check):
                    return fnames_out, planes
                
                #if not overwriting and SOME files already exist, remove just those files from the list of files to be processed
                else:
                    for fname in fnames_out:
                        if os.path.isfile(file_out):
                            fnames_out.remove(fname)
            
            
            isx.de_interleave(obj.previous_step_filepath, fnames_out, planes)
            return fnames_out, planes
        
        except:
            print(f'there was a problem running {step}.')
            traceback.print_exc()
            obj.failed = True
        return fnames_out, planes
    
    def append_name_general(self, input_name, append_str):
        vals = input_name.split('.')
        return vals[0] + '_' + append_str + '.' + vals[1]
    
    def get_output_filepath(self, obj, step):
        '''take in the current step and return a suitable filename'''
        if step not in obj.file_mods.keys():
            print(f'"{step}" not in file_modification lookup, dictionary. Will add the string that was passed.')
            append_string = step
        else:
            append_string = obj.file_mods[step]
        file_in = obj.previous_step_filepath
        path, fname = os.path.split(file_in)
        if obj.output_dir != path:
            file_out = os.path.join(obj.output_dir, fname)
            file_out = self.append_name_general(file_out,append_string)
        else:
            file_out = self.append_name_general(file_in, append_string)
        return file_out
    
    
    
    def _separate_movie(self, obj, 
                        indices, 
                        keep = [], 
                        explicit_fname = None, 
                        overwrite = True):
        '''separate a movie'''
        raise Exception('separate movie not properly implemented')
        step = 'separate_movie'
        obj.check_step(step)
        trim_arrays, fname_array = self.get_trim_array(obj,indices, keep)
        

            
        self.print_processing(obj, step)
        
        #lets just make super clear this var is only for a select case
        if explicit_fname:
            file_iterator = 0
            new_fname_array = []
            
        for trim_arr, file_out in zip(trim_arrays, fname_array):
            
            if explicit_fname:
                file_out = self.append_name_general(explicit_fname, str(file_iterator))
                file_iterator += 1
                new_fname_array += [file_out]
            
            '''this is not ready yet, '''
            if overwrite:
                if os.path.isfile(file_out):
                    print(f'overwriting previous file at {file_out}')
                    os.remove(file_out)
                else:
                    return file_out
                try: 
                    file_in = obj.previous_step_filepath
                    isx.trim_movie(file_in, file_out,trim_arr )
                except:
                    print(f'there was a problem running {step}.')
                    traceback.print_exc()
                    obj.failed = True
            
            #replace the original filename array with the one created from the 
            #explicit filename
            if explicit_fname:
                fname_array = new_fname_array
            
        return fname_array
    
    def _run_cnmfe(self, obj, processing_directory, 
                   cell_diameter = 20,
                min_corr = 0.8,
                min_pnr = 10,
                bg_spatial_subsampling = 2,
                ring_size_factor = 1.4,
                gaussian_kernel_size = 0,
                closing_kernel_size = 0,
                merge_threshold = 0.7,
                processing_mode = 'parallel_patches',
                num_threads = 4,
                patch_size = 80,
                patch_overlap = 20,
                output_unit_type = 'df_over_noise',
                explicit_fname = None, 
                overwrite = True): 
                
        step = 'cnmfe'
        obj.check_step(step)
        
        file_out = self.get_output_filepath(obj,step) if not explicit_fname else explicit_fname
        

        
        
        if os.path.isfile(file_out):
            if overwrite:
                print(f'overwriting previous file at {file_out}')
                os.remove(file_out)
            else:
                print(f'skipping processing and utilizing previous file at {file_out}')
                return file_out

        file_in = obj.previous_step_filepath
        
        self.print_processing(obj, step)
        
        isx.run_cnmfe(input_movie_files=file_in,
                    output_cell_set_files=file_out,
                    output_dir=processing_directory,
                    cell_diameter = cell_diameter,
                        min_corr = min_corr,
                        min_pnr = min_pnr,
                        bg_spatial_subsampling = bg_spatial_subsampling,
                        ring_size_factor = ring_size_factor,
                        gaussian_kernel_size = gaussian_kernel_size,
                        closing_kernel_size = closing_kernel_size,
                        merge_threshold =  merge_threshold,
                        processing_mode = processing_mode,
                        num_threads = num_threads,
                        patch_size = patch_size,
                        patch_overlap = patch_overlap,
                        output_unit_type = output_unit_type,)
        return file_out
        
    def _motion_correct(self, obj,
                        max_translation=20,
                        low_bandpass_cutoff=0.004,
                        high_bandpass_cutoff=0.016,
                        roi=None,
                        reference_segment_index=0,
                        reference_frame_index=0,
                        reference_file_name='',
                        global_registration_weight=1.0,
                        output_translation_files=None,
                        output_crop_rect_file=None,
                        explicit_fname = None,
                        overwrite = True):
        
        step = 'motion_correct'
        obj.check_step(step)
        
        file_out = self.get_output_filepath(obj,step) if not explicit_fname else explicit_fname
        
        #need to make some output files for translation csv and output translation file csv
        head, tail = os.path.split(file_out)
        fname = tail.split('.')[0]
        output_translation_file =  os.path.join(head, (fname+'translations'+'.csv'))
        output_crop_rect_file =  os.path.join(head, (fname+'mc_crop'+'.csv'))
        
        if os.path.isfile(file_out):
            if overwrite:
                print(f'overwriting previous file at {file_out}')
                os.remove(file_out)
            else:
                print(f'skipping processing and utilizing previous file at {file_out}')
                return file_out

        file_in = obj.previous_step_filepath
        
        self.print_processing(obj, step)
        
        isx.motion_correct(file_in,
                            file_out,
                            max_translation=max_translation,
                            low_bandpass_cutoff=low_bandpass_cutoff,
                            high_bandpass_cutoff=high_bandpass_cutoff,
                            roi=roi,
                            reference_segment_index=reference_segment_index,
                            reference_frame_index=reference_frame_index,
                            reference_file_name=reference_file_name,
                            global_registration_weight=global_registration_weight,
                            output_translation_files=output_translation_file,
                            output_crop_rect_file=output_crop_rect_file,
                            )
        return file_out

In [3]:
class Video_Pipe:
    pf = pipeline_functions()
    def __init__(self, input_path, session_info_file, 
                 output_dir = None, overwrite = True):
        
        #if True, reprocess and overwrite found files
        self.overwrite = overwrite
        
        with open(session_info_file) as f:
            session_info_json = json.load(f)
        self.session_info = session_info_json
        self.name = self.session_info['name']
        self.efocus = None
        self.saved_location = None
        self.previous_step_id = 0
        self.source_video_path = input_path
        
        input_dir, fname = os.path.split(input_path) 
        self.original_video_name = fname
        self.input_dir = input_dir
        self.output_dir = output_dir if output_dir else input_dir
        
        self.default_order = {'preprocess':1, 'de_interleave':0.5, 'spatial_filter':2, 
                              'separate_movie':2.5,
                              'motion_correct':3,
                              'dff':4, 'pca_ica':5, 'cnmfe':5.5, 'categorize_cells':6,
                              'longitudinal_registration':6.5, 
                             'multiplane_registration':7.5}
        self.file_mods = {'preprocess':'pp', 'spatial_filter':'sf', 'motion_correct':'mc',
                              'dff':'dff', 'separate_movie':'frames', 'pca_ica':'pca_ica', 'cnmfe':'cnmfe'}
        
        #if we want to automate the order of processing, and useful for warning the user
        #if processing is happening outside of the expected order
        self.processing_order = self.default_order
        #reverse lookup of order (ie 1:'preprocess')
        self.order_lookup = {self.default_order[key]:key for key in sorted(self.default_order.keys())}

        
        first_sub = SubPipe(self, 1)
        first_sub.previous_step_filepath = self.source_video_path
        self.subordinates = {1:first_sub}
        self.cleanup_files = []
        

    def check_status(self):
        sub0 = self.subordinates[1]
        subs = self.get_bottom_subs(sub0)
        for sub in subs:
            print('*****************************')
            print(f'SubPipe {sub.sub_ID} is at step {self.order_lookup[sub.previous_step_id]}')
            print(f'the most recent file is\n{sub.previous_step_filepath}')
            print(f'files to be removed:\n{self.cleanup_files}')
            print('*****************************\n')
            
    def get_bottom_subs(self, sub, flatten = True):
        next_processing_subs = []
        if sub.children:
            for child in sub.children:
                if child.children:
                    next_processing_subs += [self.get_bottom_subs(child)]
                else:
                    next_processing_subs +=[child]
        else:
            next_processing_subs +=[sub]
        if flatten:
            return [val for val in self.flatten_sublist(next_processing_subs)]
        else:
            return next_processing_subs
        
    def preprocess(self, sub_list = None, 
                   temporal_downsample_factor=2,
                    spatial_downsample_factor=1,
                    crop_rect=None,
                    fix_defective_pixels=True,
                    trim_early_frames=True,
                    explicit_fname = None, 
                    remove_file_on_cleanup = True,
                    overwrite = True):
        
        step = 'preprocess'
        if not sub_list:
            sub_list = self.get_bottom_subs(self.subordinates[1])
            
        for sub in sub_list:  
            
            try:
                
                fpath = self.pf._preprocess(sub,
                               temporal_downsample_factor=temporal_downsample_factor,
                                spatial_downsample_factor=spatial_downsample_factor,
                                crop_rect=crop_rect,
                                fix_defective_pixels=fix_defective_pixels,
                                trim_early_frames=trim_early_frames,
                                explicit_fname = explicit_fname,
                                overwrite=overwrite)
                
            except:
                print(f'there was a problem running {step} on SubPipe ID: {sub.sub_ID}.')
                traceback.print_exc()
                sub.failed = True
            else:
                sub.update_processing_step(step = 'preprocess', file_path = fpath)
            
                if remove_file_on_cleanup:
                    self.cleanup_files.append(fpath)
            
            self.check_failed(step)
            
            

    def check_failed(self, step):
        failed_list = []
        failed_dict = {}
        for k in self.subordinates.keys():
            sub = self.subordinates[k]
            failed_list += [sub.failed]
            failed_dict[k] = sub.failed
        
        if all(failed_list):
            print(f'all subordinates have failed at {step}')
            return True, failed_dict
        elif any(failed_list):
            print(f'some subordinates have failed {step}:')
            print(failed_dict)
            return False, failed_dict
        else:
            return False, failed_dict
        
    
    def spatial_filter(self, sub_list = None, 
                       low_cutoff=0.005,
                        high_cutoff=0.5,
                        retain_mean=False,
                        subtract_global_minimum=True,
                       explicit_fname = None, 
                       remove_file_on_cleanup = True,
                       overwrite = True):
        
        step = 'spatial_filter'

        if not sub_list:
            sub_list = self.get_bottom_subs(self.subordinates[1])
            
        for sub in sub_list:

            if not sub.failed:
                try:
                    fpath = self.pf._spatial_filter(sub, 
                                low_cutoff=low_cutoff,
                                high_cutoff=high_cutoff,
                                retain_mean=retain_mean,
                                subtract_global_minimum=subtract_global_minimum,
                                explicit_fname = explicit_fname,
                                overwrite=overwrite)
                    
                except:
                    print(f'SubPipe {sub.sub_ID} has failed at {step}')
                    sub.failed = True
                else:
                    sub.update_processing_step(step, fpath)
                    if remove_file_on_cleanup:
                        self.cleanup_files.append(fpath)
            
        self.check_failed(step)
        
        
    def append_name_general(self,input_name, append_str):
        vals = input_name.split('.')
        return vals[0] + '_' + append_str + '.' + vals[1]
    
    def append_vid_name(self,append_str):
        vals = self.video_name('.')
        return vals[0] + '_' + append_str + '.' + vals[1]
    

    def new_sub(self):
        ID = max(self.subordinates.keys()) + 1
        s = SubPipe(self, ID)
        self.subordinates[ID] = s
        return s

    
    def all_subs_failed(self):
        '''check if all subordinates have failed'''
        failed = [self.subordinates_failed[k] for k in self.subordinates_failed.keys()]
        return all(failed)['microscope']['multiplane']

        out = []
        multi = self.session_info['microscope']['multiplane']['planes']
        for key in sorted(multi.keys()):
            if multi[key]['enabled']:
                out+=[ int(multi[key]['focus'])]
        return out
    
    def save(self, file, update_save_location = True):
        
        #create a temp file so that if there is an issue saving we dont
        #overwrite our saved object
        path, _ = os.path.split(file)
        temp = os.path.join(path,'temp_pickle')
        try:
            
            with open(temp, 'wb') as f:
                self.saved_location = file
                pickle.dump(self, f)
        except:
            
            print('couldnt save this VideoPipe!')
            traceback.print_exc()
            
            #remove the empty file
            os.remove(temp)
        else:
            #cleanup by rename the temp file to the users fname
            os.rename(temp, file)
            
            if update_save_location:
                self.saved_location = file
    
    def load(self, file):
        """i wonder if this will expload?"""
        with open(file, 'rb') as f:
            return pickle.load(f)
        
    def reload(self):
        """i wonder if this will expload?"""
        if not self.saved_location:
            raise Exception('There is no saved file associated with this VideoPipe instance')
        with open(self.saved_location, 'rb') as f:
            self = pickle.load(f)
    
    def get_focal_planes(self):

        out = []
        multi = self.session_info['microscope']['multiplane']['planes']
        for key in sorted(multi.keys()):
            out+=[ int(multi[key]['focus'])]
        return out
           
    def de_interleave(self, sub_list = None, 
                      remove_file_on_cleanup = True,
                      overwrite = True):
        step = 'de_interleave'
        
        if not sub_list:
            sub_list = self.get_bottom_subs(self.subordinates[1])
        
        for sub in sub_list:
            try:
                fnames, planes = self.pf._de_interleave(sub,overwrite=overwrite)
                print(fnames)
                print(planes)
                
            except:
                print(f'there was a problem running {step} on SubPipe ID: {sub.sub_ID}.\n{traceback.print_exc()}')
                
                self.failed = True
            else:
                for fname, plane in zip(fnames, planes):
                    
                    new_sub = self.new_sub()
                    new_sub.parent_sub = sub
                    new_sub.efocus = plane
                    new_sub.sub_type = 'single_plane'
                    new_sub.update_processing_step('de_interleave', fname)
                    sub.children+= [new_sub]
                    
                    if remove_file_on_cleanup:
                        self.cleanup_files.append(fname)
                    
                
    def motion_correct(self, sub_list = None, 
                       max_translation=20,
                        low_bandpass_cutoff=0.004,
                        high_bandpass_cutoff=0.016,
                        roi=None,
                        reference_segment_index=0,
                        reference_frame_index=20,
                        reference_file_name='',
                        global_registration_weight=1.0,
                        output_translation_files=None,
                        output_crop_rect_file=None,
                        explicit_fname = None,
                        overwrite = True):
        
        step = 'motion_correct'

        if not sub_list:
            sub_list = self.get_bottom_subs(self.subordinates[1])
            
        for sub in sub_list:

            if not sub.failed:
                try:
                    fpath = self.pf._motion_correct(sub, 
                                max_translation=max_translation,
                                low_bandpass_cutoff=low_bandpass_cutoff,
                                high_bandpass_cutoff=high_bandpass_cutoff,
                                roi=roi,
                                reference_segment_index=reference_segment_index,
                                reference_frame_index=reference_frame_index,
                                reference_file_name=reference_file_name,
                                global_registration_weight=global_registration_weight,
                                output_translation_files=output_translation_files,
                                output_crop_rect_file=output_crop_rect_file,
                                explicit_fname = explicit_fname,
                                overwrite=overwrite)
                
                except:
                    traceback.print_exc()
                    print(f'SubPipe {sub.sub_ID} has failed at {step}')
                    sub.failed = True
                else:
                    sub.update_processing_step(step, fpath)
            
        self.check_failed(step)
        
        
    def dff(self, sub_list = None, f0type = 'mean', explicit_fname = None, overwrite = True):
        '''available dff types = mean, min'''
        step = 'dff'

        if not sub_list:
            sub_list = self.get_bottom_subs(self.subordinates[1])

        for sub in sub_list:

            if not sub.failed:
                try:
                    fpath = self.pf._dff(sub, f0type = f0type, 
                                         explicit_fname = explicit_fname)

                except:
                    traceback.print_exc()
                    print(f'SubPipe {sub.sub_ID} has failed at {step}')
                    sub.failed = True
                else:
                    sub.update_processing_step(step, fpath)

        self.check_failed(step)
    
    def pca_ica(self, sub_list = None,
                num_pcs=120,
                num_ics=150,
                unmix_type='spatial',
                ica_temporal_weight=0.2,
                max_iterations=100,
                convergence_threshold=1e-05,
                block_size=2000, explicit_fname = None,
                overwrite = True):
        '''HEY! 
        you need to optimize this a bit for NAc, specifically taking a look at the 
        ica temporal weight'''
        
        step = 'pca_ica'
        
        if not sub_list:
            sub_list = self.get_bottom_subs(self.subordinates[1])

        for sub in sub_list:

            if not sub.failed:
                try:
                    fpath = self.pf._pca_ica(sub,
                                            num_pcs=num_pcs,
                                            num_ics=num_ics,
                                            unmix_type=unmix_type,
                                            ica_temporal_weight=ica_temporal_weight,
                                            max_iterations=max_iterations,
                                            convergence_threshold=convergence_threshold,
                                            block_size=block_size, 
                                             explicit_fname = None,
                                             overwrite=overwrite)

                except:
                    traceback.print_exc()
                    print(f'SubPipe {sub.sub_ID} has failed at {step}')
                    sub.failed = True
                else:
                    sub.update_processing_step(step, fpath)

        self.check_failed(step)

    def run_cnmfe(self, sub_list = None,
                cell_diameter = 13,
                min_corr = 0.8,
                min_pnr = 10,
                bg_spatial_subsampling = 2,
                ring_size_factor = 1.4,
                gaussian_kernel_size = 0,
                closing_kernel_size = 0,
                merge_threshold = 0.7,
                processing_mode = 'parallel_patches',
                num_threads = 4,
                patch_size = 80,
                patch_overlap = 20,
                output_unit_type = 'df_over_noise',
                explicit_fname = None,
                overwrite = True):
        
        
        step = 'cnmfe'
        
        if not sub_list:
            sub_list = self.get_bottom_subs(self.subordinates[1])

        for sub in sub_list:

            if not sub.failed:
                try:
                    new_dir_name = f'{sub.sub_ID}_{sub.efocus}'
                    file_in = sub.previous_step_filepath
                    path, fname = os.path.split(file_in)
                    processing_directory = os.path.join(path, new_dir_name)
                    if not os.path.isdir(processing_directory):
                        os.mkdir(processing_directory)
                    fpath = self.pf._run_cnmfe(sub,
                                               processing_directory=processing_directory,
                                            cell_diameter = cell_diameter,
                                            min_corr = min_corr,
                                            min_pnr = min_pnr,
                                            bg_spatial_subsampling = bg_spatial_subsampling,
                                            ring_size_factor = ring_size_factor,
                                            gaussian_kernel_size = gaussian_kernel_size,
                                            closing_kernel_size = closing_kernel_size,
                                            merge_threshold =  merge_threshold,
                                            processing_mode = processing_mode,
                                            num_threads = num_threads,
                                            patch_size = patch_size,
                                            patch_overlap = patch_overlap,
                                            output_unit_type = output_unit_type,
                                            explicit_fname = explicit_fname,
                                            overwrite=overwrite)

                except:
                    traceback.print_exc()
                    print(f'SubPipe {sub.sub_ID} has failed at {step}')
                    sub.failed = True
                else:
                    sub.update_processing_step(step, fpath)

        self.check_failed(step)

    def flatten_sublist(self, slist):
        '''create a generator to quickly flatten lists of lists of... n lists.
        used to flatten the output of "get_bottom_subs" into a convenient interable
        
        for example: 
        lol = [[1,2,3], [ [4,5],[6,7] ], 8]
        output = [i for i in vp.flatten_sublist(lol)]
        print(output)
        >>>[1,2,3,4,5,6,7,8]
        '''
        for element in slist:
            if isinstance(element, list):
                yield from self.flatten_sublist(element)
            else:
                yield element
    
    def clean_up(self):
        _, failed_dict = self.check_failed()
        any_failed = any([v for k,v in failed_dict.items()])
        for file in self.cleanup_files:
            if os.path.isfile(file):
                try:
                    os.remove(file)
                except:
                    print('error removing file:\n{file}')
                else:
                    self.cleanup_files.remove(file)
                
            else:
                print(f'tried to cleanup file, but it did not exist:\n{file}')

    def cleanup_cnmfe_mem_maps(self,):
        '''in progress shutil.rmtree("path_to_dir")'''
                
                
class SubPipe:
    pf = pipeline_functions()
    def __init__(self, parent_pipeline, ID, parent_sub = None):
        self.efocus = None
        self.parent_pipe = parent_pipeline
        self.subtype = ''
        self.sub_ID = ID
        self.video_name = parent_pipeline.original_video_name
        self.default_order = parent_pipeline.default_order
        self.file_mods = parent_pipeline.file_mods
        self.order_lookup = parent_pipeline.order_lookup
        self.input_dir = parent_pipeline.input_dir
        self.session_info = parent_pipeline.session_info
        self.output_dir = parent_pipeline.output_dir
        self.previous_step_filepath = None
        self.previous_step_id = parent_sub.previous_step_id if parent_sub else parent_pipeline.previous_step_id
        
        self.failed = False
        
        self.children = []
    
    def update_processing_step(self, step, file_path):
        self.previous_step_filepath = file_path
        self.previous_step_id = self.parent_pipe.default_order[step]
        
    def get_focal_planes(self):
        return self.parent_pipe.get_focal_planes()
    
    def check_step(self,step):
        self.pf._check_step(self, step)
        

In [40]:
'''test_file = '/home/dprotter/Documents/Scratch Data Analysis/pipe_test_data/Session-20201102-141754_3558_operant/2020-11-02-14-40-34_video.isxd'
session_file = '/home/dprotter/Documents/Scratch Data Analysis/pipe_test_data/Session-20201102-141754_3558_operant/session.json'   
output_dir = '/home/dprotter/Documents/Scratch Data Analysis/pipe_test_data/python_output'
vp = Video_Pipe(test_file, session_info_file = session_file, output_dir = output_dir)'''

"test_file = '/home/dprotter/Documents/Scratch Data Analysis/pipe_test_data/Session-20201102-141754_3558_operant/2020-11-02-14-40-34_video.isxd'\nsession_file = '/home/dprotter/Documents/Scratch Data Analysis/pipe_test_data/Session-20201102-141754_3558_operant/session.json'   \noutput_dir = '/home/dprotter/Documents/Scratch Data Analysis/pipe_test_data/python_output'\nvp = Video_Pipe(test_file, session_info_file = session_file, output_dir = output_dir)"

In [10]:
'''import time
start = time.time()
file = '/mnt/working_storage/spring 2022/Session-20220204-140455_4345_imagePPT/2022-02-04-14-48-32_video.isxd'
output = '/mnt/working_storage/spring 2022/4345_project/4345/python_output'
session_file = '/mnt/working_storage/spring 2022/Session-20220204-140455_4345_imagePPT/session.json'

vp = Video_Pipe(file, session_info_file = session_file, output_dir = output)

vp.de_interleave()
print(f'preprocessed in {time.time() - start} seconds')
vp.preprocess(temporal_downsample_factor=1)
vp.spatial_filter()
vp.motion_correct()
vp.run_cnmfe()'''


In [ ]:
save_file = '/mnt/working_storage/spring 2022/4345_project/4345/python_output/4345.pipeline'
vp.save(save_file)


"" not in file_modification lookup, dictionary. Will add the string that was passed.
there was a problem running de_interleave.
['/mnt/working_storage/spring 2022/4345_project/4345/python_output/2022-02-09-12-49-27_video__efocus_700.isxd', '/mnt/working_storage/spring 2022/4345_project/4345/python_output/2022-02-09-12-49-27_video__efocus_1000.isxd', '/mnt/working_storage/spring 2022/4345_project/4345/python_output/2022-02-09-12-49-27_video__efocus_700.isxd']
['700', '1000', 700]
ID: 2 efocus: 700 is at step: preprocess


Traceback (most recent call last):
  File "<ipython-input-28-5b5d196335a7>", line 225, in _de_interleave
    isx.de_interleave(obj.previous_step_filepath, fnames_out, planes)
  File "/home/dprotter/Applications/Inscopix Data Processing 1.8.0/Inscopix Data Processing.linux/Contents/API/Python/isx/algo.py", line 67, in de_interleave
    efocus_arr = isx._internal.list_to_ctypes_array(in_efocus_values, ctypes.c_uint16)
  File "/home/dprotter/Applications/Inscopix Data Processing 1.8.0/Inscopix Data Processing.linux/Contents/API/Python/isx/_internal.py", line 51, in list_to_ctypes_array
    array[i] = s
TypeError: an integer is required (got type str)


some subordinates have failed preprocess:
{1: True, 2: False, 3: False, 4: False}
ID: 3 efocus: 1000 is at step: preprocess
there was a problem running preprocess on SubPipe ID: 3.
some subordinates have failed preprocess:
{1: True, 2: False, 3: True, 4: False}
overwriting previous file at /mnt/working_storage/spring 2022/4345_project/4345/python_output/2022-02-09-12-49-27_video__efocus_700_pp.isxd


Traceback (most recent call last):
  File "<ipython-input-45-d58360a92e3e>", line 95, in preprocess
    overwrite=overwrite)
  File "<ipython-input-28-5b5d196335a7>", line 56, in _preprocess
    trim_early_frames=trim_early_frames)
  File "/home/dprotter/Applications/Inscopix Data Processing 1.8.0/Inscopix Data Processing.linux/Contents/API/Python/isx/algo.py", line 46, in preprocess
    crop_rect[0], crop_rect[1], crop_rect[2], crop_rect[3], fix_defective_pixels, trim_early_frames)
  File "/home/dprotter/Applications/Inscopix Data Processing 1.8.0/Inscopix Data Processing.linux/Contents/API/Python/isx/_internal.py", line 111, in _standard_errcheck
    def _standard_errcheck(return_code, func, args=None):
KeyboardInterrupt


ID: 4 efocus: 700 is at step: preprocess


In [38]:
vp.get_bottom_subs(vp.subordinates[1])[0].get_focal_planes()

AttributeError: 'SubPipe' object has no attribute 'get_focal_planes'

In [24]:
'''file_list = ['/mnt/working_storage/synch/Session-20220405-162135_4664_baseplate/2022-04-05-16-40-58_video.isxd',
'/mnt/working_storage/synch/Session-20220405-165748_4665_baseplate/2022-04-05-17-19-14_video.isxd',
'/mnt/working_storage/synch/Session-20220407-144223_4659_ig_baseplate/2022-04-07-15-01-04_video.isxd',
'/mnt/working_storage/synch/Session-20220407-141746_4660_ig_baseplate/2022-04-07-14-29-07_video.isxd'
]

session_file_list = ['/mnt/working_storage/synch/Session-20220405-162135_4664_baseplate/session.json',
'/mnt/working_storage/synch/Session-20220405-165748_4665_baseplate/session.json',
'/mnt/working_storage/synch/Session-20220407-144223_4659_ig_baseplate/session.json',
'/mnt/working_storage/synch/Session-20220407-141746_4660_ig_baseplate/session.json'
]
'''
file_list = [
'/mnt/working_storage/synch/Session-20220407-141746_4660_ig_baseplate/2022-04-07-14-29-07_video.isxd'
]

session_file_list = [
'/mnt/working_storage/synch/Session-20220407-141746_4660_ig_baseplate/session.json'
]


output = '/mnt/working_storage/synch/pipeline_files'


for file, session_file in zip(file_list, session_file_list):
    vp = Video_Pipe(file, session_info_file = session_file, output_dir = output)
    if vp.session_info['microscope']['multiplane']['enabled']:
        vp.de_interleave(overwrite=False)
    vp.preprocess(temporal_downsample_factor=1, overwrite=False)
    
    vp.spatial_filter(overwrite=False)
    
    vp.motion_correct(overwrite=False)
    
    vp.run_cnmfe(overwrite=False)
    vp.check_status()
    #vp.clean_up()

"" not in file_modification lookup, dictionary. Will add the string that was passed.
['/mnt/working_storage/synch/pipeline_files/2022-04-07-14-29-07_video__efocus_300.isxd', '/mnt/working_storage/synch/pipeline_files/2022-04-07-14-29-07_video__efocus_500.isxd', '/mnt/working_storage/synch/pipeline_files/2022-04-07-14-29-07_video__efocus_700.isxd']
[300, 500, 700]
skipping processing and utilizing previous file at /mnt/working_storage/synch/pipeline_files/2022-04-07-14-29-07_video__efocus_300_pp.isxd
skipping processing and utilizing previous file at /mnt/working_storage/synch/pipeline_files/2022-04-07-14-29-07_video__efocus_500_pp.isxd
skipping processing and utilizing previous file at /mnt/working_storage/synch/pipeline_files/2022-04-07-14-29-07_video__efocus_700_pp.isxd
skipping processing and utilizing previous file at /mnt/working_storage/synch/pipeline_files/2022-04-07-14-29-07_video__efocus_300_pp_sf.isxd
skipping processing and utilizing previous file at /mnt/working_storage/syn

In [25]:
vp.clean_up()

TypeError: check_failed() missing 1 required positional argument: 'step'

In [14]:
vp.subordinates[1]